# Une mesure de l'inflation liée aux journaux - Projet Python pour la Data-science

### Auteurs : Lise Marchal, Raphaël Pereira et Raphaël Zambélli--Palacio

Ce notebook a pour objectif de présenter les travaux de recherche effectués dans le cadre du cours de Projet Python pour la data-science de la 2A ENSAE.

## Problématique :

Les périodes caractérisées par un taux d'inflation élevé coïcident-elles avec celles de traitement médiatique accru de cette thématique ?


# 1- Statistiques descriptives des mesures d'inflation sur la période

## 1.1- Récupération et nettoyage des données d'inflation de la FED de Saint-Louis

### 1.1.1- Utilisation de l'API de la FRED:

lien documentation pour obtenir les séries d'inflation: https://fred.stlouisfed.org/docs/api/fred/series_observations.html

In [48]:
# Création de l'URL permettant d'accéder à la série:  Consumer Price Index for All Urban Consumers: All Items in U.S. City Average

api_root="https://api.stlouisfed.org/fred/series/observations"
series_id="CPIAUCNS"
api_key="69c5f0ad2e33d8dac6297c8b3f25bb1f"
file_type="json"

url_api = (
    f"{api_root}?"
    + f"series_id={series_id}&"
    + f"api_key={api_key}&"
    + f"file_type={file_type}"
)

In [49]:
#Requête

import requests

req = requests.get(url_api)
wb = req.json()

In [50]:
#Première exploration des données
for observation in wb["observations"]:
    print(f"Date: {observation['date']}, Valeur: {observation['value']}")

Date: 1913-01-01, Valeur: 9.800
Date: 1913-02-01, Valeur: 9.800
Date: 1913-03-01, Valeur: 9.800
Date: 1913-04-01, Valeur: 9.800
Date: 1913-05-01, Valeur: 9.700
Date: 1913-06-01, Valeur: 9.800
Date: 1913-07-01, Valeur: 9.900
Date: 1913-08-01, Valeur: 9.900
Date: 1913-09-01, Valeur: 10.000
Date: 1913-10-01, Valeur: 10.000
Date: 1913-11-01, Valeur: 10.100
Date: 1913-12-01, Valeur: 10.000
Date: 1914-01-01, Valeur: 10.000
Date: 1914-02-01, Valeur: 9.900
Date: 1914-03-01, Valeur: 9.900
Date: 1914-04-01, Valeur: 9.800
Date: 1914-05-01, Valeur: 9.900
Date: 1914-06-01, Valeur: 9.900
Date: 1914-07-01, Valeur: 10.000
Date: 1914-08-01, Valeur: 10.200
Date: 1914-09-01, Valeur: 10.200
Date: 1914-10-01, Valeur: 10.100
Date: 1914-11-01, Valeur: 10.200
Date: 1914-12-01, Valeur: 10.100
Date: 1915-01-01, Valeur: 10.100
Date: 1915-02-01, Valeur: 10.000
Date: 1915-03-01, Valeur: 9.900
Date: 1915-04-01, Valeur: 10.000
Date: 1915-05-01, Valeur: 10.100
Date: 1915-06-01, Valeur: 10.100
Date: 1915-07-01, Valeur

In [51]:
#données imbriquées?
print(req.content[:1000])
# je dirais que oui, metadata


b'{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","observation_start":"1600-01-01","observation_end":"9999-12-31","units":"lin","output_type":1,"file_type":"json","order_by":"observation_date","sort_order":"asc","count":1343,"offset":0,"limit":100000,"observations":[{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-01-01","value":"9.800"},{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-02-01","value":"9.800"},{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-03-01","value":"9.800"},{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-04-01","value":"9.800"},{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-05-01","value":"9.700"},{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-06-01","value":"9.800"},{"realtime_start":"2024-12-12","realtime_end":"2024-12-12","date":"1913-07-01","value":"9.900"},{"realtime_start":"2024-12-12","realtime_end":"202

### 1.1.2- Nettoyage des données

#### Création et première exploration du dataframe

In [3]:
# Import des bibliothèques utiles: 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
# Création d'un dataframe à partir du fichier json

CPI_Urban=pd.json_normalize(wb["observations"]) #nested data?

In [54]:
CPI_Urban.head()

,realtime_start,realtime_end,date,value
0,2024-12-12,2024-12-12,1913-01-01,9.800
1,2024-12-12,2024-12-12,1913-02-01,9.800
2,2024-12-12,2024-12-12,1913-03-01,9.800
3,2024-12-12,2024-12-12,1913-04-01,9.800
4,2024-12-12,2024-12-12,1913-05-01,9.700


In [55]:
# On ne garde que les deux dernières colonnes
CPI_Urban = CPI_Urban[["date", "value"]]
CPI_Urban.head()


,date,value
0,1913-01-01,9.800
1,1913-02-01,9.800
2,1913-03-01,9.800
3,1913-04-01,9.800
4,1913-05-01,9.700


In [56]:
# Infos générales
CPI_Urban.axes

[RangeIndex(start=0, stop=1343, step=1),
 Index(['date', 'value'], dtype='object')]

In [57]:
# Infos générales: types
CPI_Urban.dtypes

date     object
value    object
dtype: object

#### Traitement des values

In [58]:
CPI_Urban['value'] = pd.to_numeric(CPI_Urban['value'], errors='coerce')


#### Traitement des dates

In [59]:
CPI_Urban['date'] = pd.to_datetime(CPI_Urban['date'])
CPI_Urban['year'] = CPI_Urban['date'].dt.year
CPI_Urban['month'] = CPI_Urban['date'].dt.month

CPI_Urban.head(15)

,date,value,year,month
0,1913-01-01,9.8,1913,1
1,1913-02-01,9.8,1913,2
2,1913-03-01,9.8,1913,3
3,1913-04-01,9.8,1913,4
4,1913-05-01,9.7,1913,5
5,1913-06-01,9.8,1913,6
6,1913-07-01,9.9,1913,7
7,1913-08-01,9.9,1913,8
8,1913-09-01,10.0,1913,9
9,1913-10-01,10.0,1913,10


#### Création de la variable pourcentage d'inflation annuelle: 

Le jeu de données se présente en effet sous la forme d'indices avec une base 100 en 1983. 
Limite: Il n'est pas idéal d'avoir la date de la base postérieure à la période étudiée.  

In [60]:
index_14=CPI_Urban.index[CPI_Urban['date'] == '1914-01-01'][0]
index_end=CPI_Urban.index[CPI_Urban['date'] == '1963-01-01'][0]

print(index_14, index_end)


12 600


In [61]:
#On ne calcule le taux d'inflation annuel que pour la période considérée
CPI_Urban['inf_an']=0.0

for i in range(index_14,index_end+1):
    b=CPI_Urban.loc[i,'value']
    a=CPI_Urban.loc[i-12,'value']
    CPI_Urban.at[i, 'inf_an']=((b-a)/a)*100

  
CPI_Urban.head(20)

,date,value,year,month,inf_an
0,1913-01-01,9.8,1913,1,0.000000
1,1913-02-01,9.8,1913,2,0.000000
2,1913-03-01,9.8,1913,3,0.000000
3,1913-04-01,9.8,1913,4,0.000000
4,1913-05-01,9.7,1913,5,0.000000
5,1913-06-01,9.8,1913,6,0.000000
6,1913-07-01,9.9,1913,7,0.000000
7,1913-08-01,9.9,1913,8,0.000000
8,1913-09-01,10.0,1913,9,0.000000
9,1913-10-01,10.0,1913,10,0.000000


#### Création d'un sous-tableau de la période considérée

In [62]:
CPI_sub = CPI_Urban[(CPI_Urban['year'] > 1913) & (CPI_Urban['year'] < 1963)]
CPI_sub.head(10)
CPI_sub.tail(10)

,date,value,year,month,inf_an
590,1962-03-01,30.1,1962,3,1.006711
591,1962-04-01,30.2,1962,4,1.342282
592,1962-05-01,30.2,1962,5,1.342282
593,1962-06-01,30.2,1962,6,1.342282
594,1962-07-01,30.3,1962,7,1.000000
595,1962-08-01,30.3,1962,8,1.337793
596,1962-09-01,30.4,1962,9,1.333333
597,1962-10-01,30.4,1962,10,1.333333
598,1962-11-01,30.4,1962,11,1.333333
599,1962-12-01,30.4,1962,12,1.333333


## 1.2- Traitement des données

### 1.2.1- Premières statistiques descriptives sur l'inflation pendant la période

In [63]:
CPI_sub['inf_an'].describe()

count    588.000000
mean       2.517532
std        6.607607
min      -15.789474
25%       -0.578035
50%        1.449275
75%        3.764927
max       23.668639
Name: inf_an, dtype: float64

In [64]:
#import seaborn as sns
#sns.lineplot(x=CPI_sub["date"],y=CPI_sub["inf_an"], marker='o')
#plt.axhline(y=0, color='r', linestyle='--', label='Ligne à y=25')

In [65]:
import plotly
import plotly.express as px
import plotly.io as pio

In [66]:
#from plotly.offline import init_notebook_mode
#init_notebook_mode(connected=True)


In [67]:
# Création du graphique (informations en passant la souris) EN FAIRE UNE FONCTION SI ON FAIT PLUSIEURS GRAPHIQUES DU MEME TYPE
# Rajouter légende

graph = px.line(CPI_sub, x="date", y="inf_an", title="Pourcentage d'inflation annuelle de 1914 à 1962")

# Légende et format des axes
graph.update_xaxes(title="Date", tickformat="%m-%Y")
graph.update_yaxes(title="Inflation (%)")

# Style du graphique
graph.update_layout(template="plotly_dark")

# Commentaires: max, min, moyenne, médiane
max=CPI_sub["inf_an"].max() 
min=CPI_sub["inf_an"].min()
mean=CPI_sub["inf_an"].mean()
median=CPI_sub["inf_an"].median()

date_max=CPI_sub.loc[CPI_sub["inf_an"] == max, "date"].values[0]
date_min=CPI_sub.loc[CPI_sub["inf_an"] == min, "date"].values[0]

# Pour l'instant l'ajout d'anotations fige le côté interactif du graphique
#graph.add_annotation(x=date_max, y=max, text="Maximum", showarrow=True, xref="x", yref="y") 
#graph.add_annotation(x=date_min, y=min, text="Minimum", showarrow=True)

graph.add_hline(y=mean, line_dash="dot", line_color="yellow", annotation_text="Moyenne", annotation_position="top left", annotation_font_color="yellow")
graph.add_hline(y=median, line_dash="dot", line_color="white", annotation_text="Médiane", annotation_position="bottom right", annotation_font_color="white")
graph.add_hline(y=0, line_color="red")

# Afficher le graphique avec zoom possible et barre à outil
graph.show(renderer="browser",config={"scrollZoom": True, "displayModeBar": True}) #pour l'instant je n'ai pas réussi à résoudre le pb pour que ça s'affiche directement dans le notebook

### 1.2.2- Identification de périodes d'intérêt

#### Identification des périodes d'inflation négative

In [68]:
# Création d'une indicatrice inf_neg qui prend la valeur 1 si l'inflation est négative et 0 sinon
CPI_sub['inf_neg']=0
CPI_sub.loc[CPI_sub['inf_an'] < 0, 'inf_neg'] = 1
CPI_sub.head(30)

C:\Users\lisem\AppData\Local\Temp\ipykernel_14432\1995190950.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,value,year,month,inf_an,inf_neg
12,1914-01-01,10.0,1914,1,2.040816,0
13,1914-02-01,9.9,1914,2,1.020408,0
14,1914-03-01,9.9,1914,3,1.020408,0
15,1914-04-01,9.8,1914,4,0.000000,0
16,1914-05-01,9.9,1914,5,2.061856,0
17,1914-06-01,9.9,1914,6,1.020408,0
18,1914-07-01,10.0,1914,7,1.010101,0
19,1914-08-01,10.2,1914,8,3.030303,0
20,1914-09-01,10.2,1914,9,2.000000,0
21,1914-10-01,10.1,1914,10,1.000000,0


In [69]:
"""
graph = px.line(CPI_sub, x="date", y="inf_an", title="Pourcentage d'inflation annuelle de 1914 à 1962",animation_frame="year", animation_group="inf_neg")
# Légende et format des axes
graph.update_xaxes(title="Date", tickformat="%m-%Y",range=[1914-01-1,1963-12-01])
graph.update_yaxes(title="Inflation (%)",range=[-25,25])

# Style du graphique
graph.update_layout(template="plotly_dark")

# Commentaires
graph.add_annotation(x="1920-06-01", text="Maximum d'inflation sur la période", showarrow=True)
graph.show(renderer="browser")
"""

'\ngraph = px.line(CPI_sub, x="date", y="inf_an", title="Pourcentage d\'inflation annuelle de 1914 à 1962",animation_frame="year", animation_group="inf_neg")\n# Légende et format des axes\ngraph.update_xaxes(title="Date", tickformat="%m-%Y",range=[1914-01-1,1963-12-01])\ngraph.update_yaxes(title="Inflation (%)",range=[-25,25])\n\n# Style du graphique\ngraph.update_layout(template="plotly_dark")\n\n# Commentaires\ngraph.add_annotation(x="1920-06-01", text="Maximum d\'inflation sur la période", showarrow=True)\ngraph.show(renderer="browser")\n'

In [70]:
# Premières informations sur les périodes d'inflation négatives: 
CPI_neg=CPI_sub[CPI_sub['inf_neg'] == 1]
CPI_neg['inf_an'].describe()




count    158.000000
mean      -4.149776
std        4.029451
min      -15.789474
25%       -7.680995
50%       -2.228430
75%       -0.980392
max       -0.371747
Name: inf_an, dtype: float64

#### Identification des périodes d'accélération et de descélération

Différence relative par rapport au pourcentage d'inflation d'il y a un an.
A discuter: je pense qu'il est plus intéressant de regarder la différence relative. Pb: des fois on a des pourcentages d'inflation nulle. Ce sont les valeurs infinies dans le tableau. Soit je les laisse telles quelles, soit je les remplace par une valeur (je propose le max de acceleration_12?)

In [102]:
#peut-être à utiliser dans la création du pourcentage d'inflation
# .loc permet de gérer le statut de copie de CPI_sub
#je voulais faire en relatif mais problème avec les pourcentages négatifs (ln) ou nuls(b-a/a)
CPI_sub.loc[:, 'acceleration_12'] = (CPI_sub['inf_an']/CPI_sub['inf_an'].shift(12) - 1)*100 # peut-être
CPI_sub.head(30)
#quand valeur il y a 12 mois=0 inf


,date,value,year,month,inf_an,inf_neg,acceleration_12
12,1914-01-01,10.0,1914,1,2.040816,0,NaN
13,1914-02-01,9.9,1914,2,1.020408,0,NaN
14,1914-03-01,9.9,1914,3,1.020408,0,NaN
15,1914-04-01,9.8,1914,4,0.000000,0,NaN
16,1914-05-01,9.9,1914,5,2.061856,0,NaN
17,1914-06-01,9.9,1914,6,1.020408,0,NaN
18,1914-07-01,10.0,1914,7,1.010101,0,NaN
19,1914-08-01,10.2,1914,8,3.030303,0,NaN
20,1914-09-01,10.2,1914,9,2.000000,0,NaN
21,1914-10-01,10.1,1914,10,1.000000,0,NaN


Graphique des pourcentages d'accélération (sans avoir remplacé les valeurs infinies)

In [72]:
# Création du graphique (sans avoir remplacé les valeurs infinies)
graph = px.line(CPI_sub, x="date", y="acceleration_12", title="Pourcentage d'accélération de l'inflation annuelle par rapport à sa valeur il y a un an de 1915 à 1962")

# Légende et format des axes
graph.update_xaxes(title="Date", tickformat="%m-%Y")
graph.update_yaxes(title="Accélération (%)")

# Style du graphique
graph.update_layout(template="plotly_dark")

# Commentaires: max, min, moyenne, médiane

meanp=CPI_sub.loc[CPI_sub["acceleration_12"] > 0, "acceleration_12"].mean()
meann=CPI_sub.loc[CPI_sub["acceleration_12"]<0,"acceleration_12"].mean()
medianp=CPI_sub.loc[CPI_sub["acceleration_12"]>0,"acceleration_12"].median()
mediann=CPI_sub.loc[CPI_sub["acceleration_12"]<0,"acceleration_12"].median()


# Pour l'instant l'ajout d'anotations fige le côté interactif du graphique
#graph.add_annotation(x=date_max, y=max, text="Maximum", showarrow=True, xref="x", yref="y") 
#graph.add_annotation(x=date_min, y=min, text="Minimum", showarrow=True)

graph.add_hline(y=meanp, line_dash="dot", line_color="yellow", annotation_text="Accélération moyenne", annotation_position="top left", annotation_font_color="yellow")
graph.add_hline(y=medianp, line_dash="dot", line_color="white", annotation_text="Accélération médiane", annotation_position="bottom right", annotation_font_color="white")
graph.add_hline(y=meann, line_dash="dot", line_color="yellow", annotation_text="Décélération moyenne", annotation_position="top left", annotation_font_color="yellow")
graph.add_hline(y=mediann, line_dash="dot", line_color="white", annotation_text="Décélération médiane", annotation_position="bottom right", annotation_font_color="white")
#graph.add_hline(y=0, line_color="red")

# Afficher le graphique avec zoom possible et barre à outil
graph.show(renderer="browser",config={"scrollZoom": True, "displayModeBar": True}) #pour l'instant je n'ai pas réussi à résoudre le pb pour que ça s'affiche directement dans le notebook

Graphiques en remplaçant les valeurs infinies par la moyenne de la valeur précédente et de la valeur suivante

In [103]:
# moyenne valeur d'avant et après

# On le fait un nb arbitraire de fois pour traiter les cas de 3,4,5,6 valeurs infinies se suivant. 
# On ne fait pas une boucle while par exemple sur la moyenne pour éviter les boucles infinies

for i in range(5):
    # traitement des valeurs infinies isolées: remplacement par la moyenne entre la valeur précédente et la valeur suivante
    CPI_sub.loc[
        abs(CPI_sub["acceleration_12"]) == np.inf,
        "acceleration_12"
    ] = (CPI_sub["acceleration_12"].shift(1) + CPI_sub["acceleration_12"].shift(-1)) / 2
    #traitement des valeurs infinies suivies de valeurs infinies: remplacement par la valeur précédente
    CPI_sub.loc[
        (abs(CPI_sub["acceleration_12"]) == np.inf) & (abs(CPI_sub["acceleration_12"].shift(-1)) == np.inf),
        "acceleration_12"
    ] = CPI_sub["acceleration_12"].shift(1)

    #traitement des valeurs infinies précédées de valeurs infinies: remplacement par la valeur suivante
    CPI_sub.loc[
        (abs(CPI_sub["acceleration_12"]) == np.inf) & (abs(CPI_sub["acceleration_12"].shift(1)) == np.inf),
        "acceleration_12"
    ] = CPI_sub["acceleration_12"].shift(-1)

CPI_sub.head(20)



,date,value,year,month,inf_an,inf_neg,acceleration_12
12,1914-01-01,10.0,1914,1,2.040816,0,NaN
13,1914-02-01,9.9,1914,2,1.020408,0,NaN
14,1914-03-01,9.9,1914,3,1.020408,0,NaN
15,1914-04-01,9.8,1914,4,0.000000,0,NaN
16,1914-05-01,9.9,1914,5,2.061856,0,NaN
17,1914-06-01,9.9,1914,6,1.020408,0,NaN
18,1914-07-01,10.0,1914,7,1.010101,0,NaN
19,1914-08-01,10.2,1914,8,3.030303,0,NaN
20,1914-09-01,10.2,1914,9,2.000000,0,NaN
21,1914-10-01,10.1,1914,10,1.000000,0,NaN


In [104]:
CPI_sub["acceleration_12"].describe()

count     571.000000
mean      -34.855874
std       326.991413
min     -2309.282700
25%      -119.275362
50%       -41.016949
75%        31.073446
max      1587.943262
Name: acceleration_12, dtype: float64

In [105]:
# Création du graphique (sans avoir remplacé les valeurs infinies)
graph = px.line(CPI_sub, x="date", y="acceleration_12", title="Pourcentage d'accélération de l'inflation annuelle par rapport à sa valeur il y a un an de 1915 à 1962")

# Légende et format des axes
graph.update_xaxes(title="Date", tickformat="%m-%Y")
graph.update_yaxes(title="Accélération (%)")

# Style du graphique
graph.update_layout(template="plotly_dark")

# Commentaires: max, min, moyenne, médiane

meanp=CPI_sub.loc[CPI_sub["acceleration_12"] > 0, "acceleration_12"].mean()
meann=CPI_sub.loc[CPI_sub["acceleration_12"]<0,"acceleration_12"].mean()
medianp=CPI_sub.loc[CPI_sub["acceleration_12"]>0,"acceleration_12"].median()
mediann=CPI_sub.loc[CPI_sub["acceleration_12"]<0,"acceleration_12"].median()


# Pour l'instant l'ajout d'anotations fige le côté interactif du graphique
#graph.add_annotation(x=date_max, y=max, text="Maximum", showarrow=True, xref="x", yref="y") 
#graph.add_annotation(x=date_min, y=min, text="Minimum", showarrow=True)

graph.add_hline(y=meanp, line_dash="dot", line_color="yellow", annotation_text="Accélération moyenne", annotation_position="top left", annotation_font_color="yellow")
graph.add_hline(y=medianp, line_dash="dot", line_color="white", annotation_text="Accélération médiane", annotation_position="bottom right", annotation_font_color="white")
graph.add_hline(y=meann, line_dash="dot", line_color="yellow", annotation_text="Décélération moyenne", annotation_position="top left", annotation_font_color="yellow")
graph.add_hline(y=mediann, line_dash="dot", line_color="white", annotation_text="Décélération médiane", annotation_position="bottom right", annotation_font_color="white")
#graph.add_hline(y=0, line_color="red")

# Afficher le graphique avec zoom possible et barre à outil
graph.show(renderer="browser",config={"scrollZoom": True, "displayModeBar": True}) #pour l'instant je n'ai pas réussi à résoudre le pb pour que ça s'affiche directement dans le notebook

# 2- Analyse de fréquence sur le corpus de presse

## 2.1- Création des fichiers de fréquence

## 2.2- Statistiques descriptives sur les fréquences

In [6]:
# Récupération des données: voir si adaptation une fois les emrges effectués

freq_40 = pd.read_csv("frequences_data_1940_1949.csv")
freq_40.head()

,key,nbre_articles,inflation,disinflation,inflationary,deflation,prices,cost,wages,currency,...,consumption basket,purchasing power,consumer price index,rise in prices,fall in prices,cost of living,inflation expectations,money supply growth,central bank policy,economic price adjustments
0,1940-01,40090,4,0,3,1,778,1557,184,81,...,0,27,0,1,0,11,0,0,0,0
1,1940-02,41579,3,0,2,1,760,1534,177,83,...,0,23,0,1,0,8,0,0,0,0
2,1940-03,47118,0,0,0,1,852,1769,198,75,...,0,18,0,1,0,7,0,0,0,0
3,1940-04,44607,2,0,1,0,802,1622,154,88,...,0,18,0,2,0,5,0,0,0,0
4,1940-05,43840,3,0,2,2,818,1879,216,53,...,0,10,0,1,0,15,0,0,0,0
